In [7]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0, "../src/")
sys.path.insert(0, "../../../src/")
sys.path.insert(0, "../../../projects/skimpy_bench/src/")
sys.path.insert(0, "../../../projects/skimpy_blox/")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import skimpy
import skimpy.functional as F
import skimpy_3d
import skimpy.config as skimpy_config
from pythreejs import *
import numpy as np
from IPython.display import display

In [9]:
def blur(tensor):
    G_x = (1 / 64.0) * skimpy.Tensor.from_list([[[1, 6, 15, 20, 15, 6, 1]]]).to(float)
    G_y = (1 / 64.0) * skimpy.Tensor.from_list([[[1], [6], [15], [20], [15], [6], [1]]]).to(float)
    G_z = (1 / 64.0) * skimpy.Tensor.from_list([[[1]], [[6]], [[15]], [[20]], [[15]], [[6]], [[1]]]).to(float)
    tensor = F.conv_3d(tensor, G_x, padding=(3, 0, 0), fill=0.0)
    tensor = F.conv_3d(tensor, G_y, padding=(0, 3, 0), fill=0.0)  
    tensor = F.conv_3d(tensor, G_z, padding=(0, 0, 3), fill=0.0)
    return tensor

In [10]:
def render(tensor):
    config = skimpy_3d.VoxelConfig()
    config[0] = skimpy_3d.EmptyVoxel()
    config[1] = skimpy_3d.ColorVoxel(0, 200, 200)
    mesh = skimpy_3d.generate_mesh(config, tensor._tensor)
    js_mesh = Mesh(
        geometry=BufferGeometry(
            attributes={
                "position": BufferAttribute(
                    np.array(mesh.positions, dtype='float32').reshape(-1, 3),
                    normalized=False,
                ),
                "normal": BufferAttribute(
                    np.array(mesh.normals, dtype='float32').reshape(-1, 3),
                    normalized=False,
                ),
                "color": BufferAttribute(
                    np.array(mesh.colors, dtype='uint8').reshape(-1, 4)
                ),
                "index": BufferAttribute(
                    np.array(mesh.triangles, dtype='uint32'),
                    normalized=False,
                ),
            },
        ),
        material=MeshLambertMaterial(vertexColors="VertexColors"),
        position=[0, 0, 0],
    )
    return js_mesh

def render_marching_cubes(tensor):
    mesh = skimpy_3d.marching_cubes(tensor._tensor, 0.5)
    geometry = BufferGeometry(
        attributes={
            "position": BufferAttribute(
                np.array(mesh.positions, dtype='float32').reshape(-1, 3),
                normalized=False,
            ),
            "index": BufferAttribute(
                np.array(mesh.triangles, dtype='uint32'),
                normalized=False,
            ),
        },
    )

    geometry.exec_three_obj_method("computeVertexNormals")
    js_mesh = Mesh(
        geometry=geometry,
        material=MeshLambertMaterial(color="#00c8c8"),
        position=[0, 0, 0],
    )
    return js_mesh

In [11]:
import skimpy.config as pyskip_config
import skimpy as pyskip

In [12]:
with pyskip_config.lazy_evaluation_scope():
    big_cube = pyskip.Tensor(shape=(200, 200, 200), val=0, dtype=int)
    big_rect = pyskip.Tensor(shape=(50, 50, 200), val=0, dtype=int)
    big_cube[:, :, :] = 1
    big_rect[:, :, :] = 1

    big_cube[30:80, 120:170, : ] -= big_rect
    big_cube[120:170, 120:170, : ] -= big_rect
    
    big_cube[50:100, 40:90, : ] -= big_rect
    big_cube[100:150, 40:90, : ] -= big_rect
    
    big_cube[50:100, 50:80:4, : ] += big_rect[::, 10:40:4, :]
    big_cube[100:150, 50:80:4, : ] += big_rect[::, 10:40:4, :]
    
    
print("Compression: %.0fx" % ((200 ** 3) / (2 * big_cube.rle_length())))
render(big_cube)

Compression: 70x


Preview(child=Mesh(geometry=BufferGeometry(attributes={'position': <BufferAttribute shape=(2713600, 3), dtype=…

In [13]:
with pyskip_config.lazy_evaluation_scope():
    big_cube = pyskip.Tensor(shape=(200, 200, 200), val=0, dtype=int)
    big_rect = pyskip.Tensor(shape=(50, 50, 200), val=0, dtype=int)
    big_cube[:, :, :] = 1
    big_rect[:, :, :] = 1

    big_cube[30:80, 120:170, : ] -= big_rect
    big_cube[120:170, 120:170, : ] -= big_rect
    
    big_cube[50:100, 40:90, : ] -= big_rect
    big_cube[100:150, 40:90, : ] -= big_rect
    
    big_cube[50:100, 50:80:4, : ] += big_rect[::, 10:40:4, :]
    big_cube[100:150, 50:80:4, : ] += big_rect[::, 10:40:4, :]
    
    

box_blur_kernel = pyskip.Tensor((10,10,10), val=1 / (10 **3), dtype=float)
big_cube = skimpy.convolve.pad_3d_tensor(big_cube.to(float), 10, 0)
blurry_cube = skimpy.convolve.conv_3d(big_cube, box_blur_kernel)
    
print("Compression: %.0fx" % ((200 ** 3) / (2 * blurry_cube.rle_length())))
render_marching_cubes(blurry_cube)

Compression: 2x


Preview(child=Mesh(geometry=BufferGeometry(attributes={'position': <BufferAttribute shape=(365700, 3), dtype=f…

In [14]:
print("Compression: %.0fx" % ((200 ** 3) / (2 * blurry_cube.rle_length())))
render_marching_cubes(blurry_cube)

Compression: 2x


Preview(child=Mesh(geometry=BufferGeometry(attributes={'position': <BufferAttribute shape=(365700, 3), dtype=f…

In [15]:
print("Compression: %.0fx" % ((200 ** 3) / (2 * big_cube.rle_length())))
render(big_cube)

Compression: 29x


TypeError: generate_mesh(): incompatible function arguments. The following argument types are supported:
    1. (arg0: _skimpy_3d_cpp_ext.VoxelConfig, arg1: _skimpy_cpp_ext.Tensor3i) -> _skimpy_3d_cpp_ext.VoxelMesh

Invoked with: <_skimpy_3d_cpp_ext.VoxelConfig object at 0x7f1d8c1c9fb0>, Tensor<3, f>([0.0, 0.0, 0.0, 0.0, ..., 0.0])

In [ ]:
config = skimpy_3d.VoxelConfig()
config[0] = skimpy_3d.EmptyVoxel()
config[1] = skimpy_3d.ColorVoxel(0, 200, 200)

In [ ]:
%%time
mesh = skimpy_3d.generate_mesh(config, (t > 0.5).to(int)._tensor)

In [ ]:
from pythreejs import *
import numpy as np
from IPython.display import display

In [ ]:
js_mesh = Mesh(
    geometry=BufferGeometry(
        attributes={
            "position": BufferAttribute(
                np.array(mesh.positions, dtype='float32').reshape(-1, 3),
                normalized=False,
            ),
            "normal": BufferAttribute(
                np.array(mesh.normals, dtype='float32').reshape(-1, 3),
                normalized=False,
            ),
            "color": BufferAttribute(
                np.array(mesh.colors, dtype='uint8').reshape(-1, 4)
            ),
            "index": BufferAttribute(
                np.array(mesh.triangles, dtype='uint32'),
                normalized=False,
            ),
        },
    ),
    material=MeshLambertMaterial(vertexColors="VertexColors"),
    position=[0, 0, 0],
)

In [ ]:
js_mesh

In [ ]:
%%time
for i in range(2):
    t = sum([
        0.125 * t[0::2, 0::2, 0::2],
        0.125 * t[1::2, 0::2, 0::2],
        0.125 * t[0::2, 1::2, 0::2],
        0.125 * t[1::2, 1::2, 0::2],
        0.125 * t[0::2, 0::2, 1::2],
        0.125 * t[1::2, 0::2, 1::2],
        0.125 * t[0::2, 1::2, 1::2],
        0.125 * t[1::2, 1::2, 1::2],
    ])

In [ ]:
avg_kernel = skimpy.Tensor.from_list([[[1, 1], [1, 1]], [[1, 1], [1, 1]]]).to(float) / 8
density_lattice = F.conv_3d(
    t,
    avg_kernel,
    padding=1,
)

In [ ]:
%%time
mesh = skimpy_3d.marching_cubes(density_lattice._tensor, 0.5)

In [ ]:
geometry = BufferGeometry(
        attributes={
            "position": BufferAttribute(
                np.array(mesh.positions, dtype='float32').reshape(-1, 3),
                normalized=False,
            ),
            "index": BufferAttribute(
                np.array(mesh.triangles, dtype='uint32'),
                normalized=False,
            ),
        },
    )

geometry.exec_three_obj_method("computeVertexNormals")

In [ ]:
js_mesh = Mesh(
    geometry=geometry,
    material=MeshLambertMaterial(color="blue"),
    position=[0, 0, 0],
)

In [ ]:
js_mesh

In [ ]:
wf_geometry = WireframeGeometry(geometry)

In [ ]:
js_lines = LineSegments(
    geometry=wf_geometry,
    material=LineBasicMaterial(color="green"),
    position=[0, 0, 0],
)

In [ ]:
js_lines

In [ ]:
camera = PerspectiveCamera(position=(64, 64, 64), lookAt=(0,0,0), fov=20)
scene = Scene(children=[js_mesh, camera])

In [ ]:
renderer = Renderer(
    camera=camera, 
    background="white", 
    background_opacity=1,
    scene=scene,
    controls=[OrbitControls(controlling=camera)],
    width=984,
    height=984,
)

display(renderer)